# **Importing Dependencies**

In [ ]:
!pip install geonamescache
!pip install unidecode
!pip install cartopy

In [ ]:
from geonamescache import GeonamesCache
from unidecode import unidecode
import regex as re
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
gc = GeonamesCache()
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
import cartopy
from cartopy.crs import PlateCarree
import math

# **Finding Longitude & Latitude of headline location from extracted cities**

---



Loading previously formed data frame with headline, city, country

In [ ]:
df_country_city = pd.read_csv(
    '/content/df_headline_city_country.csv',
    header=0
)

In [ ]:
df_country_city

,Headlines,City,Country
0,Zika Outbreak Hits Miami,Miami,NaN
1,Could Zika Reach New York City?,New York City,NaN
2,First Case of Zika in Miami Beach,Miami Beach,NaN
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
4,Dallas man comes down with case of Zika,Dallas,NaN
...,...,...,...
601,Rumors about Rabies spreading in Jerusalem hav...,Jerusalem,NaN
602,More Zika patients reported in Indang,Indang,NaN
603,Suva authorities confirmed the spread of Rotav...,Suva,NaN
604,More Zika patients reported in Bella Vista,Vista,NaN


# **Finding city names which have multiple cities**

In [ ]:
cities_name_found = [city for city in df_country_city['City']]
multiple_cities = [city for city in cities_name_found
                    if (len(gc.get_cities_by_name(city))>1)]
print(f"{len(multiple_cities)} names have multiple cities of a same name")

175 names have multiple cities of a same name




# **Fetching Country Code**









In [ ]:
country_code_present = []
for keys in cities.keys():
  for i in range(len(cities_name_found)):
    if (unidecode(cities[keys]['name'])==cities_name_found[i]):
      if (len(gc.get_cities_by_name(cities_name_found[i]))>1):
        df_country_city.loc[df_country_city['City'] == cities_name_found[i],'Country_Code'] = None
      else:
        df_country_city.loc[df_country_city['City'] == cities_name_found[i],'Country_Code'] = cities[keys]['countrycode']
        country_code_present.append(cities[keys]['countrycode'])

# **Country codes present in the headlines**

In [ ]:
unique_country_code_present = set(country_code_present)

Country Code that are present in data without considering city name with multiple cities

In [ ]:
print(unique_country_code_present)

{'NZ', 'RU', 'ZW', 'IT', 'PR', 'IL', 'DK', 'CN', 'AO', 'GE', 'SA', 'MA', 'US', 'CG', 'BS', 'PA', 'AZ', 'AU', 'TW', 'PF', 'KE', 'VI', 'GA', 'EC', 'UG', 'VN', 'CA', 'ML', 'MH', 'GT', 'CV', 'AM', 'ID', 'IR', 'FR', 'NI', 'GB', 'KP', 'SV', 'CZ', 'DO', 'CU', 'SN', 'MM', 'HN', 'FJ', 'ZA', 'SG', 'ZM', 'CD', 'CL', 'MY', 'OM', 'VE', 'SI', 'PE', 'BZ', 'UY', 'JP', 'PK', 'HK', 'BE', 'PT', 'NP', 'CH', 'CO', 'SE', 'IN', 'MX', 'AR', 'NG', 'BR', 'EG', 'TZ', 'SD', 'TH', 'CM', 'KR', 'ES', 'ET', 'PY', 'DE', 'KH', 'YE', 'PH', 'BD', 'AE', 'TT'}


# Distrubution of country code

In [ ]:
grouped = df_country_city.groupby(['Country_Code'])


In [ ]:
country_code_distribution = {}
for key in grouped.groups.keys():
  country_code_distribution[key] = len(grouped.get_group(key))
sorted_country_code_distribution = sorted(country_code_distribution.items(), key=lambda x: x[1] ,reverse=True)


In [ ]:
country_codes_list = [country_code for country_code, count in sorted_country_code_distribution]
print(country_codes_list) # US have highest number of instances

['US', 'BR', 'PH', 'IN', 'CO', 'MX', 'MY', 'CA', 'CN', 'DE', 'FR', 'ZA', 'AU', 'GB', 'ID', 'TH', 'JP', 'CH', 'FJ', 'KH', 'MM', 'NG', 'SV', 'TT', 'TZ', 'VN', 'AZ', 'BZ', 'DK', 'EC', 'NI', 'PA', 'PE', 'PT', 'SN', 'TW', 'UG', 'ZM', 'AE', 'AM', 'AO', 'AR', 'BE', 'BS', 'CD', 'CG', 'CM', 'CU', 'CV', 'CZ', 'DO', 'EG', 'ET', 'GA', 'GE', 'GT', 'HK', 'HN', 'IL', 'IR', 'IT', 'KE', 'KP', 'KR', 'MA', 'MH', 'ML', 'NP', 'NZ', 'OM', 'PF', 'PK', 'PR', 'PY', 'RU', 'SA', 'SD', 'SE', 'SG', 'SI', 'UY', 'VE', 'VI', 'YE', 'ZW']


# **Adding Longitude and Latitude to dataframe**

*   Values for city name with multiple cities based on populations



In [ ]:
population_of_multiple_cities = []
for keys in cities.keys():
  for i in range(len(cities_name_found)):
    if (cities_name_found[i] == unidecode(cities[keys]['name'])):
        print("city name is", cities_name_found[i])
        if (len(gc.get_cities_by_name(cities_name_found[i]))>1):
          population_of_multiple_cities = []
          test = gc.get_cities_by_name(cities_name_found[i])
          for index, value in enumerate(test):
              for key in test[index].keys():
                  # print("geonameid index : ", test[index][key]['geonameid'],"population", test[index][key]['population'])
                  population_of_multiple_cities.append((test[index][key]['geonameid'],test[index][key]['population']))
                  # print(population_of_multiple_cities)
                  population_of_multiple_cities = sorted(population_of_multiple_cities, key=lambda x: x[1] ,reverse=True)
                  # print(population_of_multiple_cities)

                  # max_population_city_key = cities[key]['latitude']
              max_population_city_key = str(population_of_multiple_cities[0][0])
              df_country_city.loc[df_country_city['City'] == cities_name_found[i],'Country_Code'] = cities[max_population_city_key]['countrycode']
              df_country_city.loc[df_country_city['City'] == cities_name_found[i],'latitude'] = cities[max_population_city_key]['latitude']
              df_country_city.loc[df_country_city['City'] == cities_name_found[i],'longitude'] = cities[max_population_city_key]['longitude']
        else:

         df_country_city.loc[df_country_city['City'] == cities_name_found[i],'latitude'] = cities[keys]['latitude']
         df_country_city.loc[df_country_city['City'] == cities_name_found[i],'longitude'] = cities[keys]['longitude']


In [ ]:
df_country_city

,Headlines,City,Country,Country_Code,latitude,longitude
0,Zika Outbreak Hits Miami,Miami,NaN,US,25.77427,-80.19366
1,Could Zika Reach New York City?,New York City,NaN,US,40.71427,-74.00597
2,First Case of Zika in Miami Beach,Miami Beach,NaN,US,25.79065,-80.13005
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil,BR,-8.05389,-34.88111
4,Dallas man comes down with case of Zika,Dallas,NaN,US,32.78306,-96.80667
...,...,...,...,...,...,...
601,Rumors about Rabies spreading in Jerusalem hav...,Jerusalem,NaN,IL,31.76904,35.21633
602,More Zika patients reported in Indang,Indang,NaN,PH,14.19528,120.87694
603,Suva authorities confirmed the spread of Rotav...,Suva,NaN,FJ,-18.13683,178.42531
604,More Zika patients reported in Bella Vista,Vista,NaN,US,33.20004,-117.24254


In [ ]:
df_country_city[df_country_city['latitude'].isna() & df_country_city['longitude'].isna()]

,Headlines,City,Country,Country_Code,latitude,longitude


In [ ]:
df_country_city

,Headlines,City,Country,Country_Code,latitude,longitude
0,Zika Outbreak Hits Miami,Miami,NaN,US,25.77427,-80.19366
1,Could Zika Reach New York City?,New York City,NaN,US,40.71427,-74.00597
2,First Case of Zika in Miami Beach,Miami Beach,NaN,US,25.79065,-80.13005
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil,BR,-8.05389,-34.88111
4,Dallas man comes down with case of Zika,Dallas,NaN,US,32.78306,-96.80667
...,...,...,...,...,...,...
601,Rumors about Rabies spreading in Jerusalem hav...,Jerusalem,NaN,IL,31.76904,35.21633
602,More Zika patients reported in Indang,Indang,NaN,PH,14.19528,120.87694
603,Suva authorities confirmed the spread of Rotav...,Suva,NaN,FJ,-18.13683,178.42531
604,More Zika patients reported in Bella Vista,Vista,NaN,US,33.20004,-117.24254


All 606 have now latitude and longitude

# **Data frame with city, country, countrycode, latitude & longitude**

In [ ]:
df_country_city.to_csv('df_headlines_city_countrycode_lat_long_new.csv',
                       index = False)
